In [ ]:
library(data.table)
library(dplyr)
library(nparLD)
library(missForest)
library(parallel)
library(doParallel)

# HuMet DATASET

In this notebook, we analyze a subset of the HuMet dataset, focusing on **plasma** samples across three platforms:

- Metabolon HD4 (nt-ms)
- Biocrates p150 (t-ms)
- In-house biochemistry (chem.)

## Loading the Data

In [ ]:
met_data <- fread("../input/raw/humet_data_raw_none_subjects15_tp57.csv", sep = ",", fill = TRUE)
info_data <- fread("../input/humet_info.csv", sep = ",", fill = TRUE)
head(met_data)
head(info_data)

## Data Analysis

In [ ]:
# Count metabolite columns
num_metabolite_columns <- length(setdiff(names(met_data), c("time", "subject")))

unique_time_values <- unique(met_data$time)
unique_subject_values <- unique(met_data$subject)

print(paste("Number of metabolites:", num_metabolite_columns))
print(paste("Unique time values:", paste(unique_time_values, collapse = ", ")))
print(paste("Unique subject values:", paste(unique_subject_values, collapse = ", ")))

In [ ]:
# Count occurrences of each unique platform
platform_counts <- table(info_data$platform_name)
print(platform_counts)

- Targeted: 132
- Non-Targeted: 502
- Insulin (Hormone): 1

## Preprocessing

## Adding Challenge information

**Relevant time intervals for our analysis**:

Since the original dataset lacked challenge information, we assigned it based on the time column:

- **Fasting**: Time points 1–9
- **Physical Activity**: Time points 33–39
- **Oral Lipid Tolerance Test (OLTT)**: Time points 40–49

In [ ]:
# Create a dataset with all time intervals
met_data_all <- met_data %>%
  mutate(challenge = case_when(
    time >= 1 & time <= 9 ~ "Fasting",
    time >= 33 & time <= 39 ~ "Physical Activity",
    time >= 40 & time <= 49 ~ "OLTT",
    TRUE ~ "Other"  # Keep "Other" instead of filtering out
  ))

# Count removed rows
num_removed <- nrow(met_data_all) - nrow(met_data)
cat("Number of rows removed:", num_removed, "\n")

# Display first rows of both datasets
#tail(met_data_all)
tail(met_data)

In [ ]:
# Count occurrences of each unique challenge
challenge_counts <- table(met_data$challenge)
print(challenge_counts)

## Removing Metabolites with > 30% missing values

In [5]:
remove_high_na_metabolites <- function(met_data, threshold = 0.3, output_file = "removed_metabolites.txt") {
  # Identify metabolite columns (excluding time, subject, and challenge)
  metabolite_columns <- setdiff(colnames(met_data), c("time", "subject", "challenge"))
  
  # Calculate the percentage of missing values for each metabolite
  na_percentage <- colMeans(is.na(met_data[, ..metabolite_columns]))

  # Find metabolites with more than `threshold` missing values
  high_na_metabolites <- names(na_percentage[na_percentage > threshold])

  # Write the removed metabolite names to a text file
  if (length(high_na_metabolites) > 0) {
    writeLines(high_na_metabolites, output_file)
  }

  # Remove these metabolites from met_data
  filtered_met_data <- met_data[, !high_na_metabolites, with = FALSE]

  return(filtered_met_data)
}

# Apply function to clean met_data
met_data <- remove_high_na_metabolites(met_data)

In [ ]:

# Create a dataset with only relevant time intervals
met_data <- met_data_all %>%
  filter(challenge != "Other")

In [ ]:
tail(met_data)

## Splitting the dataset based on platform

In [7]:
# Identify metabolite columns (excluding time, subject, and challenge)
metabolite_columns <- setdiff(colnames(met_data), c("time", "subject", "challenge"))

# Define platform patterns for Metabolon and Biocrates
platforms <- list(
  metabolon = "\\[P, nt-ms\\]",
  biocrates = "\\[P, t-ms\\]"
)

# Function to filter metabolites based on platform
filter_metabolites <- function(pattern) {
  selected_cols <- c("time", "subject", "challenge", metabolite_columns[grepl(pattern, metabolite_columns)])
  met_data[, ..selected_cols]
}

# Function to filter metabolites NOT belonging to Metabolon or Biocrates (i.e., Inhouse)
filter_inhouse_metabolites <- function() {
  excluded_cols <- unique(unlist(lapply(platforms, function(p) metabolite_columns[grepl(p, metabolite_columns)])))
  selected_cols <- c("time", "subject", "challenge", setdiff(metabolite_columns, excluded_cols))
  met_data[, ..selected_cols]
}

# Create datasets
met_data_metabolon <- filter_metabolites(platforms$metabolon)
met_data_biocrates <- filter_metabolites(platforms$biocrates)
met_data_inhouse <- filter_inhouse_metabolites()  # Everything else

# Inspect results
#list(met_data_metabolon, met_data_biocrates, met_data_inhouse)


## Handle Missing Values - missForest

The dataset was downloaded with the **"Concentrations and relative abundances"** transformation applied.  
According to the HuMet documentation, the following preprocessing steps were already performed:

#### 1. **Manual Data Curation**
- Data points exceeding **4 times the standard deviation** at a given time point were flagged.
- If these outliers were **not within the first 30 minutes** of a challenge, they were considered for exclusion.
- After **manual inspection**, **92 data points** were removed.

#### 2. **Missing Data**
- The dataset **does not contain manually excluded data points**.

Since these steps were applied in the repository, **additional outlier removal and manual curation are not necessary.**

In [ ]:
# Count missing values in each dataset
sum(is.na(data_fasting))
sum(is.na(data_exercise))
sum(is.na(data_oltt))

In [ ]:
# Function to convert categorical variables to factors
convert_to_factors <- function(data) {
  data %>%
    mutate(
      challenge = as.factor(challenge),
      time = as.factor(time),
      subject = as.factor(subject)
    ) %>%
    mutate(across(where(is.character), as.factor))
}

# Function for missForest imputation with adaptive parallelization
perform_missForest <- function(data_subset, ntree_val = 10) {
  num_vars <- ncol(data_subset)  # Get the number of variables
  
  # Adjust cores to be at most the number of variables
  num_cores <- min(detectCores() - 1, num_vars)
  
  # If parallelization is still invalid, set it to 'no'
  parallel_option <- if (num_cores > 1) "variables" else "no"
  
  cl <- makeCluster(num_cores, type = "FORK") 
  registerDoParallel(cl)
  
  set.seed(42)  # Ensures reproducibility
  imputed_data <- missForest(data_subset, ntree = ntree_val, parallelize = parallel_option, verbose = TRUE)
  
  stopCluster(cl)  # Stop cluster
  
  return(imputed_data$ximp)  # Extract imputed dataset
}

# Wrapper function to process and impute metabolite datasets
data_pipeline <- function(metabolite_datasets) {
  # Convert categorical variables to factors
  metabolite_datasets <- lapply(metabolite_datasets, convert_to_factors)
  
  # Perform imputation with automatic parallelization adjustment
  imputed_data <- lapply(metabolite_datasets, perform_missForest, ntree_val = 10)
  
  return(imputed_data)
}

# List of metabolite datasets
metabolite_datasets <- list(
  metabolon = met_data_metabolon,
  biocrates = met_data_biocrates,
  inhouse = met_data_inhouse
)

# Apply pipeline to each dataset
imputed_metabolite_data <- data_pipeline(metabolite_datasets)

In [ ]:
head(imputed_metabolite_data$metabolon)

## Reformating the Table

Bringing 3 platform specific datasets together

Merging the data 

In [ ]:
# Merge met_data with info_data based on metabolite and platform_name
met_data <- merge(met_data, 
                  info_data[, .(metabolite, platform_name, super_pathway, sub_pathway)], 
                  by = c("metabolite", "platform_name"), 
                  all.x = TRUE)  # Keep all rows in met_data

# Print first rows to verify the merge
head(met_data)

In [ ]:
# View rows where Platform is 'Biocrates_p150'
filtered_data <- met_data[platform_name == "Biocrates p150 [t-ms]"]

# Print first rows
filtered_data

In [ ]:
# Function to reshape a dataset into long format and add platform information
reshape_long <- function(data, platform_name) {
  # Identify metabolite columns (exclude time, subject, challenge)
  metabolite_columns <- setdiff(names(data), c("time", "subject", "challenge"))

  # Convert all metabolite columns to numeric (preserves NA values)
  data[, (metabolite_columns) := lapply(.SD, as.numeric), .SDcols = metabolite_columns]

  # Reshape into long format
  long_data <- melt(data,
                    id.vars = c("time", "subject", "challenge"),  # Keep these columns unchanged
                    measure.vars = metabolite_columns,  # Only reshape metabolite columns
                    variable.name = "metabolite",
                    value.name = "response",
                    na.rm = FALSE)  # Keep NA values instead of removing them

  # Add platform column
  long_data[, platform_name := platform_name]

  return(long_data)
}

# Reshape all three datasets and combine them
met_data <- rbindlist(
  list(
    reshape_long(imputed_metabolite_data$metabolon, "Metabolon HD4 [nt-ms]"),
    reshape_long(imputed_metabolite_data$biocrates, "Biocrates p150 [t-ms]"),
    reshape_long(imputed_metabolite_data$inhouse, "In-house biochemistry [chem.]")
  ),
  use.names = TRUE,
  fill = TRUE
)

# Print first rows of the final combined dataset
head(met_data)

Clean up the metabolite names

In [ ]:
# Clean metabolite names by removing anything inside square brackets and trimming whitespace
met_data[, metabolite := gsub("\\[.*?\\]", "", metabolite)]  # Remove text inside brackets
met_data[, metabolite := trimws(metabolite)]  # Trim leading/trailing spaces
met_data[, metabolite := tolower(metabolite)]  # Convert to lowercase

# Print first rows to verify changes
head(met_data)


Some data is missing from our met_data dataset (e.g. super_pathway and sub_pathway), we need to add it for further analysis. For this we use info_data.

Clean up of info_data:

In [ ]:
# Keep only rows where fluid == "plasma"
info_data <- info_data[fluid == "plasma"]

# Ensure correct encoding and remove asterisks
info_data$metabolite <- gsub("[*]", "", info_data$metabolite)  # Remove all asterisks
info_data$metabolite <- gsub("\u200B", "", info_data$metabolite)  # Remove zero-width spaces (if present)
info_data$metabolite <- gsub("[[:space:]]+$", "", info_data$metabolite)  # Trim trailing spaces
info_data$metabolite <- trimws(info_data$metabolite)  # Remove any remaining spaces
info_data$metabolite <- tolower(info_data$metabolite)  # Convert to lowercase

head(info_data)

# Postprandial DATASET

## NON-IMPUTED DATASET PREPROCESSING

In [ ]:
met_data <- fread("../input/raw/postprandial_non_imputed.csv", sep = ";", fill = TRUE)
info_data <- fread("../input/raw/postprandial_info.csv", sep = ";", fill = TRUE, header=TRUE)

met_data[, V1 := NULL]
info_data[, V1 := NULL]

head(met_data)
head(info_data)

In [ ]:
# Select columns excluding challenge, challenge_time, and subject
selected_columns <- setdiff(names(met_data), c("challenge", "challenge_time", "subject"))

# Count total number of NA values in these columns
total_na_selected <- sum(is.na(met_data[, ..selected_columns]))

# Print result
print(total_na_selected)

In [ ]:
# Find columns that match "cis-aconitate" and "sebacate (decanedioate)_metabolon"
matching_columns <- names(met_data)[grepl("cis-aconitate", names(met_data), ignore.case = TRUE) |
                                    grepl("^sebacate \\(decanedioate\\)_metabolon$", names(met_data), ignore.case = TRUE)]

# Check if columns exist
if (length(matching_columns) > 0) {
  # Calculate percentage of missing values for each column
  na_percentages <- colMeans(is.na(met_data[, ..matching_columns])) * 100
  
  # Print results
  print("Percentage of missing values per column:")
  print(na_percentages)
} else {
  print("Columns not found!")
}

In [48]:
## REMOVE OVER 30% MISSINGNESS

remove_high_na_metabolites <- function(met_data, threshold = 0.3, output_file = "removed_metabolites.txt") {
  # Identify metabolite columns (excluding time, subject, and challenge)
  metabolite_columns <- setdiff(colnames(met_data), c("subject", "challenge", "challenge_time"))
  
  # Calculate the percentage of missing values for each metabolite
  na_percentage <- colMeans(is.na(met_data[, ..metabolite_columns]))

  # Find metabolites with more than `threshold` missing values
  high_na_metabolites <- names(na_percentage[na_percentage > threshold])

  # Write the removed metabolite names to a text file
  if (length(high_na_metabolites) > 0) {
    writeLines(high_na_metabolites, output_file)
  }

  # Remove these metabolites from met_data
  filtered_met_data <- met_data[, !high_na_metabolites, with = FALSE]

  return(filtered_met_data)
}

# Apply function to clean met_data
met_data <- remove_high_na_metabolites(met_data)


- after loading the data: 4465 metabolites with na values 
- after removing > 30% missingness: 4222 metabolites with na values left
- in this dataset insulin is the only metabolite in biochemistry dataset

In [ ]:
## SPLITTING THE DATA

# Identify metabolite columns (excluding time, challenge, and challenge_time)
metabolite_columns <- setdiff(colnames(met_data), c("subject", "challenge", "challenge_time"))

# Define platform patterns based on metabolite column names
platforms <- list(
  metabolon = "metabolon",
  biocrates = "biocrates",
  inhouse = "biochemistry"
)

# Function to filter metabolites based on platform
filter_metabolites <- function(pattern) {
  selected_cols <- c("subject", "challenge", "challenge_time", metabolite_columns[grepl(pattern, metabolite_columns, ignore.case = TRUE)])
  met_data[, ..selected_cols]
}

# Function to filter metabolites NOT belonging to Metabolon or Biocrates (i.e., Inhouse)
filter_inhouse_metabolites <- function() {
  excluded_cols <- unique(unlist(lapply(platforms[-3], function(p) metabolite_columns[grepl(p, metabolite_columns, ignore.case = TRUE)])))
  selected_cols <- c("subject", "challenge", "challenge_time", setdiff(metabolite_columns, excluded_cols))
  met_data[, ..selected_cols]
}

# Create datasets based on metabolite column names
met_data_metabolon <- filter_metabolites(platforms$metabolon)
met_data_biocrates <- filter_metabolites(platforms$biocrates)
met_data_inhouse <- filter_inhouse_metabolites()  # Everything else

## MISSFOREST

# Function to convert categorical variables to factors
convert_to_factors <- function(data) {
  data %>%
    mutate(
      challenge = as.factor(challenge),
      challenge_time = as.factor(challenge_time),
      subject = as.factor(subject)
    ) %>%
    mutate(across(where(is.character), as.factor))
}

# Function for missForest imputation with adaptive parallelization
perform_missForest <- function(data_subset, ntree_val = 10) {
  num_vars <- ncol(data_subset)  # Get the number of variables
  
  # Adjust cores to be at most the number of variables
  num_cores <- min(detectCores() - 1, num_vars)
  
  # If parallelization is still invalid, set it to 'no'
  parallel_option <- if (num_cores > 1) "variables" else "no"
  
  cl <- makeCluster(num_cores, type = "FORK") 
  registerDoParallel(cl)
  
  set.seed(42)  # Ensures reproducibility
  imputed_data <- missForest(data_subset, ntree = ntree_val, parallelize = parallel_option, verbose = TRUE)
  
  stopCluster(cl)  # Stop cluster
  
  return(imputed_data$ximp)  # Extract imputed dataset
}

# Wrapper function to process and impute metabolite datasets
data_pipeline <- function(metabolite_datasets) {
  # Convert categorical variables to factors
  metabolite_datasets <- lapply(metabolite_datasets, convert_to_factors)
  
  # Perform imputation with automatic parallelization adjustment
  imputed_data <- lapply(metabolite_datasets, perform_missForest, ntree_val = 10)
  
  return(imputed_data)
}

# List of metabolite datasets
metabolite_datasets <- list(
  metabolon = met_data_metabolon,
  biocrates = met_data_biocrates,
  inhouse = met_data_inhouse
)

# Apply pipeline to each dataset
imputed_metabolite_data <- data_pipeline(metabolite_datasets)

### Comparing With Imputed Data

In [ ]:
## LOADING IMPUTED DATA 

met_data_imputed <- fread("../input/raw/postprandial_imputed.csv", sep = ";", fill = TRUE)
met_data_imputed[, V1 := NULL]

## SPLITTING THE DATA

# Identify metabolite columns (excluding time, challenge, and challenge_time)
metabolite_columns <- setdiff(colnames(met_data_imputed), c("subject", "challenge", "challenge_time"))

# Define platform patterns based on metabolite column names
platforms <- list(
  metabolon = "metabolon",
  biocrates = "biocrates",
  inhouse = "biochemistry"
)

# Function to filter metabolites based on platform
filter_metabolites <- function(pattern) {
  selected_cols <- c("subject", "challenge", "challenge_time", metabolite_columns[grepl(pattern, metabolite_columns, ignore.case = TRUE)])
  met_data_imputed[, ..selected_cols]
}

# Function to filter metabolites NOT belonging to Metabolon or Biocrates (i.e., Inhouse)
filter_inhouse_metabolites <- function() {
  excluded_cols <- unique(unlist(lapply(platforms[-3], function(p) metabolite_columns[grepl(p, metabolite_columns, ignore.case = TRUE)])))
  selected_cols <- c("subject", "challenge", "challenge_time", setdiff(metabolite_columns, excluded_cols))
  met_data_imputed[, ..selected_cols]
}

# Create datasets based on metabolite column names
met_data_metabolon_imputed <- filter_metabolites(platforms$metabolon)
met_data_biocrates_imputed <- filter_metabolites(platforms$biocrates)
met_data_inhouse_imputed <- filter_inhouse_metabolites()  # Everything else

# Inspect results
#list(met_data_metabolon, met_data_biocrates, met_data_inhouse)

met_data_metabolon_imputed

## IMPUTED DATASET ADJUSTMENTS 

In [ ]:
met_data <- fread("../input/raw/postprandial_imputed.csv", sep = ";", fill = TRUE)
info_data <- fread("../input/raw/postprandial_info.csv", sep = ";", fill = TRUE, header=TRUE)

met_data[, V1 := NULL]
info_data[, V1 := NULL]

head(met_data)
head(info_data)

- 0 NA values
- In imputed postprandial dataset only 3 types of platforms are present: biocrates, biochemistry, metabolon

In [ ]:
# Identify metabolite columns (excluding subject, challenge, and challenge_time)
metabolite_columns <- setdiff(names(met_data), c("subject", "challenge_time", "challenge"))

# Convert all metabolite columns to numeric (preserves NA values)
met_data[, (metabolite_columns) := lapply(.SD, as.numeric), .SDcols = metabolite_columns]

# Reshape into long format
met_data <- melt(met_data,
                      id.vars = c("subject", "challenge_time", "challenge"),  # Keep these columns unchanged
                      measure.vars = metabolite_columns,  # Only reshape metabolite columns
                      variable.name = "metabolite",
                      value.name = "response",
                      na.rm = FALSE)  # Keep NA values instead of removing them

# Extract platform name from metabolite column (text after last underscore "_")
met_data[, platform_name := sub(".*_", "", metabolite)]

# Map extracted platform names to correct labels
met_data[, platform_name := fifelse(platform_name == "biocrates", "Biocrates p150 [t-ms]",
                                  fifelse(platform_name == "biochemistry", "In-house biochemistry [chem.]",
                                  fifelse(platform_name == "metabolon", "Metabolon HD4 [nt-ms]", NA_character_)))]

# Keep only rows where fluid == "plasma"
info_data <- info_data[Fluid == "Plasma"]

# Rename col_code in info_data to match metabolite column in met_data
setnames(info_data, "col_code", "metabolite")

# Merge met_data with info_data based on metabolite and platform_name
met_data <- merge(met_data, 
                  info_data[, .(metabolite, platform_name, SUPER.PATHWAY, SUB.PATHWAY)], 
                  by = c("metabolite", "platform_name"), 
                  all.x = TRUE)  # Keep all rows in met_data

# Rename columns properly
setnames(met_data, c("platform_name", "SUPER.PATHWAY", "SUB.PATHWAY"), 
                  c("platform", "super_pathway", "sub_pathway"))

# Remove everything after the last underscore "_" in metabolite names
met_data[, metabolite := sub("_.*", "", metabolite)]


# Reorder columns to match the desired structure
setcolorder(met_data, c("metabolite", "super_pathway", "sub_pathway", 
                         "platform", "response", "subject", "challenge_time", "challenge"))

# Print first rows to verify changes
head(met_data)


# Hypothesis Testing

635 columns, but only 634 are relevant for ANOVA, since insulin is not a metabolite

NO INSULIN RUN THIS 

In [ ]:
# Create a new dataset excluding rows where platform is "In-house biochemistry [chem.]"
met_data_filtered <- met_data[platform != "In-house biochemistry [chem.]", ]

# Print the first few rows of the new dataset
head(met_data_filtered)

INSULIN NEEDED? RUN THIS!

In [5]:
met_data_filtered <- met_data

### ANOVA-like Test

In [ ]:
# Define significance threshold after multiple testing correction
p_threshold <- 0.05 / 634

# Convert challenge_time to a categorical variable
met_data_filtered[, challenge_time := as.factor(challenge_time)]

# Subset data by challenge
metabolite_data_ogtt <- met_data_filtered[challenge == "ogtt"]
metabolite_data_oltt <- met_data_filtered[challenge == "oltt"]
metabolite_data_sld <- met_data_filtered[challenge == "sld"]

# Function to run ANOVA-like test while considering metabolite & platform
run_anova_like_test <- function(metabolite_data, challenge_name) {
    
    # Initialize results list
    results <- list()

    # Loop through unique metabolite-platform combinations
    unique_metabolites <- unique(metabolite_data[, .(metabolite, platform, super_pathway, sub_pathway)])
    
    for (i in seq_len(nrow(unique_metabolites))) {
        
        met <- unique_metabolites$metabolite[i]
        plat <- unique_metabolites$platform[i]
        super_path <- unique_metabolites$super_pathway[i]
        sub_path <- unique_metabolites$sub_pathway[i]
        
        # Subset data for this metabolite and platform
        subset_data <- metabolite_data[metabolite == met & platform == plat]
        
        # Ensure sufficient data points for analysis
        if (nrow(subset_data) > 2) {
            
            # Run the ld.f1 test
            test_result <- ld.f1(y = subset_data$response, 
                                 time = subset_data$challenge_time, 
                                 subject = subset_data$subject, 
                                 description=FALSE)

            # Extract p-value for time effect
            p_value <- test_result$ANOVA.test$`p-value`
            
            # Store results
            results[[paste(met, plat, sep = "_")]] <- data.table(
              challenge = challenge_name,
              metabolite = met,
              platform = plat,
              super_pathway = super_path,
              sub_pathway = sub_path,
              p_value = p_value
            )
        }
    }

    # Combine results into a data frame
    anova_results <- rbindlist(results, fill = TRUE)

    # Identify significant time effects
    anova_results[, significant := p_value < p_threshold]

    return(anova_results)
}

# Run ANOVA-like test for each challenge
anova_results_ogtt <- run_anova_like_test(metabolite_data_ogtt, "OGTT")
anova_results_oltt <- run_anova_like_test(metabolite_data_oltt, "OLTT")
anova_results_sld <- run_anova_like_test(metabolite_data_sld, "SLD")

# Combine all results into one table
#final_anova_results <- rbind(anova_results_ogtt, anova_results_oltt, anova_results_sld, fill = TRUE)
#final_anova_results

In [ ]:
head(anova_results_ogtt)

#### Significant effect of time on metabolite levels during at least one challenge

In [ ]:
# Get all unique metabolites from the updated dataset (excluding insulin)
all_metabolites <- unique(met_data_filtered[, .(metabolite, platform, super_pathway, sub_pathway)])

# Sort metabolites first by super_pathway, then sub_pathway, then metabolite name
all_metabolites <- all_metabolites[order(super_pathway, sub_pathway, tolower(metabolite))]

# Initialize the column as FALSE for all metabolites
all_metabolites[, significant_any_challenge := FALSE]

# Extract **only** significant metabolites (ensuring metabolite-platform pairs match)
significant_ogtt <- anova_results_ogtt[significant == TRUE, .(metabolite, platform)]
significant_oltt <- anova_results_oltt[significant == TRUE, .(metabolite, platform)]
significant_sld <- anova_results_sld[significant == TRUE, .(metabolite, platform)]

# Function to update significance status **only for matching metabolite + platform pairs**
update_significance <- function(met_data_filtered, sig_data) {
    if (nrow(sig_data) > 0) {  # Only run if there's data
        met_data_filtered[sig_data, on = .(metabolite,platform), significant_any_challenge := TRUE]
    }
}

# Update based on **corrected** significance lists
update_significance(all_metabolites, significant_ogtt)
update_significance(all_metabolites, significant_oltt)
update_significance(all_metabolites, significant_sld)

# Save the output file
#fwrite(all_metabolites, "../results/anova_results_significant_in_at_least_one_challenge.csv")

# Display output
head(all_metabolites)

### Figure 2 

In [ ]:
# Merge results into one table
anova_results_combined <- merge(anova_results_ogtt, anova_results_oltt, by = c("metabolite", "platform", "super_pathway", "sub_pathway"), all = TRUE)
anova_results_combined <- merge(anova_results_combined, anova_results_sld, by = c("metabolite", "platform", "super_pathway", "sub_pathway"), all = TRUE)

# Rename p-value columns for each challenge
setnames(anova_results_combined, 
         c("p_value.x", "p_value.y", "p_value"), 
         c("p_value_ogtt", "p_value_oltt", "p_value_sld"))

# Remove the redundant challenge columns and significance columns as they are not needed
anova_results_combined[, c("challenge.x", "challenge.y", "challenge", "significant.x", "significant.y", "significant") := NULL]

# Merge the p-values into the all_metabolites dataframe
all_metabolites_fig <- merge(all_metabolites, anova_results_combined, by = c("metabolite", "platform", "super_pathway", "sub_pathway"), all.x = TRUE)

# Filter rows where platform is either "Metabolon HD4 [nt-ms]" or "In-house biochemistry [chem.]" 
# and significant_any_challenge is TRUE
all_metabolites_fig <- all_metabolites_fig[
  platform %in% c("Metabolon HD4 [nt-ms]", "In-house biochemistry [chem.]")
] #  & significant_any_challenge == TRUE

# Remove the 'significant_any_challenge' column
#all_metabolites_fig[, significant_any_challenge := NULL]

# Merge all_metabolites with met_data based on metabolite and platform
all_metabolites_fig <- merge(all_metabolites_fig, met_data[, .(metabolite, platform, subject, challenge_time, 
                            challenge, response)], 
                                by = c("metabolite", "platform"), all.x = TRUE)
# p_value_ogtt, significant_ogtt, p_value_oltt, significant_oltt, p_value_sld, significant_sld

# Calculate the mean for each metabolite, platform, challenge, and challenge_time
all_metabolites_fig[, mean_response := mean(response, na.rm = TRUE), 
                       by = .(metabolite, platform, challenge, challenge_time)]

# Remove 'subject' and 'response' columns
all_metabolites_fig <- all_metabolites_fig[, !c("subject", "response"), with = FALSE]

# Remove duplicates based on all columns
all_metabolites_fig <- unique(all_metabolites_fig)

# Calculate log2_foldchange based on the difference in mean_response
all_metabolites_fig[, log2_foldchange := 
                      mean_response - mean_response[challenge_time == 0],
                    by = .(metabolite, platform, challenge)]

# Create a new column 'p_value' based on the 'challenge' column
all_metabolites_fig[, p_value := 
    ifelse(challenge == "ogtt", p_value_ogtt,
    ifelse(challenge == "oltt", p_value_oltt,
    ifelse(challenge == "sld", p_value_sld, NA)))] 

# Now remove the original p_value columns (p_value_ogtt, p_value_oltt, p_value_sld)
all_metabolites_fig[, c("p_value_ogtt", "p_value_oltt", "p_value_sld") := NULL]

# Create a new column 'neg_log_p_value' that takes -log10 of the 'p_value' column
all_metabolites_fig[, neg_log10_p_value := -log10(p_value)]

# Create a new column 'abs_log2_foldchange' that stores the absolute value of 'log2_foldchange'
all_metabolites_fig[, abs_log2_foldchange := abs(log2_foldchange)]

# Filter rows where either condition applies: abs_log2_foldchange > 1 or neg_log10_p_value > 40
all_metabolites_fig <- all_metabolites_fig[abs_log2_foldchange > 1  | (significant_any_challenge == TRUE & neg_log10_p_value > 40)]

# Sort the table in decreasing order of abs_log2_foldchange
setorder(all_metabolites_fig, -abs_log2_foldchange)

# Display the result
head(all_metabolites_fig)

# Save the table to a CSV file
fwrite(all_metabolites_fig, "../results/figure_2_table.csv")


#### Significant effect of time on metabolite levels during at all challenges

In [ ]:
# Initialize columns for each category as FALSE
all_metabolites[, `:=`(
  significant_OGTT = FALSE,
  significant_OLTT = FALSE,
  significant_SLD = FALSE,
  significant_OGTT_OLTT = FALSE,
  significant_OGTT_SLD = FALSE,
  significant_OLTT_SLD = FALSE,
  significant_OGTT_OLTT_SLD = FALSE
)]

# Function to update significance for each individual challenge
update_significance <- function(met_data, sig_data, column_name) {
    if (nrow(sig_data) > 0) {
        met_data[sig_data, on = .(metabolite, platform), (column_name) := TRUE]
    }
}

# Update individual significance columns
update_significance(all_metabolites, significant_ogtt, "significant_OGTT")
update_significance(all_metabolites, significant_oltt, "significant_OLTT")
update_significance(all_metabolites, significant_sld, "significant_SLD")

# Debugging: Check individual significance counts
print(paste("OGTT:", sum(all_metabolites$significant_OGTT)))
print(paste("OLTT:", sum(all_metabolites$significant_OLTT)))
print(paste("SLD:", sum(all_metabolites$significant_SLD)))

# Calculate overlaps
all_metabolites[, `:=`(
  significant_OGTT_OLTT = significant_OGTT & significant_OLTT & !significant_SLD,
  significant_OGTT_SLD = significant_OGTT & significant_SLD & !significant_OLTT,
  significant_OLTT_SLD = significant_OLTT & significant_SLD & !significant_OGTT,
  significant_OGTT_OLTT_SLD = significant_OGTT & significant_OLTT & significant_SLD
)]

# Debugging: Check overlap counts
print(paste("OGTT ∩ OLTT:", sum(all_metabolites$significant_OGTT_OLTT)))
print(paste("OGTT ∩ SLD:", sum(all_metabolites$significant_OGTT_SLD)))
print(paste("OLTT ∩ SLD:", sum(all_metabolites$significant_OLTT_SLD)))
print(paste("OGTT ∩ OLTT ∩ SLD:", sum(all_metabolites$significant_OGTT_OLTT_SLD)))

# Save the updated table
#fwrite(all_metabolites, "results/all_metabolites_with_significance.csv")

### T-Test 

In [ ]:
# Select baseline data for different challenges
baseline_ogtt <- met_data[challenge == "ogtt" & challenge_time == "0"]
baseline_sld  <- met_data[challenge == "ogtt" & challenge_time == "240"]
baseline_oltt <- met_data[challenge == "oltt" & challenge_time == "0"]

# Ensure only common subjects are used across all three conditions
common_subjects <- Reduce(intersect, list(baseline_ogtt$subject, baseline_sld$subject, baseline_oltt$subject))
baseline_ogtt <- baseline_ogtt[subject %in% common_subjects]
baseline_sld  <- baseline_sld[subject %in% common_subjects]
baseline_oltt <- baseline_oltt[subject %in% common_subjects]

# Get unique metabolite-platform combinations
metabolites <- unique(met_data[, .(metabolite, platform)])

# Perform paired t-tests and keep significance + pathway info
results <- lapply(1:nrow(metabolites), function(i) {
  met_name <- metabolites$metabolite[i]
  met_platform <- metabolites$platform[i]

  # Subset using both metabolite name and platform
  ogtt_values <- baseline_ogtt[metabolite == met_name & platform == met_platform, response]
  sld_values  <- baseline_sld[metabolite == met_name & platform == met_platform, response]
  oltt_values <- baseline_oltt[metabolite == met_name & platform == met_platform, response]

  # Ensure valid comparisons (at least two values in each group)
  if (length(ogtt_values) > 1 & length(sld_values) > 1 & length(oltt_values) > 1) {
    mean_diff_sld  <- mean(sld_values, na.rm = TRUE) - mean(ogtt_values, na.rm = TRUE)
    mean_diff_oltt <- mean(oltt_values, na.rm = TRUE) - mean(ogtt_values, na.rm = TRUE)

    # Perform paired t-tests
    p_val_sld <- tryCatch(
      t.test(sld_values, ogtt_values, paired = TRUE, var.equal = FALSE)$p.value,
      error = function(e) NA
    )
    p_val_oltt <- tryCatch(
      t.test(oltt_values, ogtt_values, paired = TRUE, var.equal = FALSE)$p.value,
      error = function(e) NA
    )

    # Retrieve significance info and pathway details

    # Should you metabolites be significant in any_challenge or in all_challenges 
    # ALL CHALLENGES: significant_OGTT_OLTT_SLD
    # ANY CHALLENGE: significant_any_challenge
    anova_significance <- all_metabolites[metabolite == met_name & platform == met_platform, significant_OGTT_OLTT_SLD]
    pathway_info <- all_metabolites[metabolite == met_name & platform == met_platform, 
                                    .(platform, super_pathway, sub_pathway, significant_OGTT_OLTT_SLD)]

    # Return structured result
    return(data.table(
      metabolite = met_name,
      platform = met_platform,
      super_pathway = pathway_info$super_pathway,
      sub_pathway = pathway_info$sub_pathway,
      mean_diff_sld = mean_diff_sld,
      p_val_sld = p_val_sld,
      mean_diff_oltt = mean_diff_oltt,
      p_val_oltt = p_val_oltt,
      significant_response = anova_significance
    ))
  } else {
    return(NULL)
  }
})

# Remove NULL results safely
results <- rbindlist(Filter(Negate(is.null), results), fill = TRUE)

# Filter rows where significant_OGTT_OLTT_SLD is TRUE
filtered_results <- results[significant_response == TRUE]

# Save filtered results to a CSV file
fwrite(filtered_results, "../results/paired_ttest_results.csv")
message("Filtered T-Test completed! Filtered results saved in: ../results/paired_ttest_results.csv")

# Print summary of the filtered results
num_significant <- nrow(filtered_results)
total_tests <- nrow(results)
percentage <- (num_significant / total_tests) * 100

cat("Number of significant metabolites (filtered):", num_significant, "\n")
cat("Percentage of significant results (filtered):", round(percentage, 2), "%\n")


In [ ]:
# Map missing information from met_data_filtered to filtered_results based on both "metabolite" and "platform"
clustering_input <- merge(
  filtered_results, 
  met_data_filtered[, .(metabolite, platform, response, subject, challenge, challenge_time)], 
  by = c("metabolite", "platform"), 
  all.x = TRUE
)

# Save the updated results
fwrite(clustering_input, "../results/clustering_input.csv")

# Display the first few rows of the updated data
head(clustering_input)

# Clustering

is already integrated into the pipeline, tested by Laura